# Cinetel Data Cleaning

Cinetel data scraped daily from their boxoffice published everyday at this [link](https://www.cinetel.it/pages/boxoffice.php?edperiodo=aWVyaQ==).

## Reading In Files

In [1]:
import pandas as pd

In [2]:
df1 = pd.read_csv(r"E:\data_analysis_python\cinetel\cinetel_.csv")
df1.head()

,Pos.,Titolo,Prima Progr.,Nazione,Distribuzione,Incasso,Presenze,Incasso al 16/11/2023,Presenze al 16/11/2023,2023-11-17 15:31:50.351708
0,1,C'E' ANCORA DOMANI,'2023-10-26'26/10/2023,ITA,VISION DISTRIBUTION,449991.06,66700,15231636.85,2202169,2023-11-17 15:31:50.351708
1,2,HUNGER GAMES - LA BALLATA DELL'USIGNOLO E DEL ...,'2023-11-15'15/11/2023,USA,MEDUSA FILM S.P.A.,238466.31,32469,584902.45,79311,2023-11-17 15:31:50.351708
2,3,THE MARVELS,'2023-11-08'08/11/2023,USA,WALT DISNEY S.M.P. ITALIA,60914.35,8628,2260142.44,305291,2023-11-17 15:31:50.351708
3,4,THANKSGIVING,'2023-11-16'16/11/2023,USA,EAGLE PICTURES S.P.A.,41232.57,5709,41232.57,5709,2023-11-17 15:31:50.351708
4,5,THE OLD OAK,'2023-11-16'16/11/2023,FRA,LUCKY RED DISTRIB.,36605.89,5879,56540.79,8609,2023-11-17 15:31:50.351708


---

## Cleaning the dataframe

To clean the dataframe I decided to drop duplicate values and columns that aren't relevant to my analysis, such as `df1['Incasso al 16/11/2023']` and `df1['Presenze al 16/11/2023]`. I then decided to rename the existing columns using the snake_case rule in order to make the analysis easier.

In [3]:
df1 = df1.drop_duplicates()

In [4]:
# df1 = df1.drop(columns= ['Incasso al 16/11/2023', 'Presenze al 16/11/2023'])

# at first i thought these columns weren't relevant
# making visualizations, i realized i need the data to have accurate insights on movies that started their screenings before i started to scrape the data

In [5]:
df1 = df1.rename(columns = {'Pos.':'daily_rank', 'Titolo' : 'title', 'Prima Progr.' : 'first_screening_date', 'Nazione' : 'nation', 'Distribuzione' : 'distribution', 'Incasso' : 'daily_takings', 'Presenze' : 'daily_attendance','Incasso al 16/11/2023' : 'total_takings',  'Presenze al 16/11/2023' : 'total_attendance', '2023-11-17 15:31:50.351708' : 'date_pulled'})

In [6]:
# dropping the extra 'header row' that is imported everyday when the .csv file gets updated
df1 = df1[df1['daily_rank'].str.contains('Pos.')==False]

In [7]:
# making the 'title' and 'distribution' entries consistent by appling the title format
df1['title'] = df1['title'].apply(lambda x: x.title())
df1['distribution'] = df1['distribution'].apply(lambda x: x.title())

In [8]:
# cleaning the 'first_screening_date' entries
df1['first_screening_date'] = df1['first_screening_date'].str[12:]

In [9]:
# cleaning the column showing the date the data was pulled from the site
df1['date_pulled'] = pd.to_datetime(df1['date_pulled'])
df1['date_pulled'] = pd.to_datetime(df1['date_pulled']).dt.date

#extra tries:
#df1['date_pulled'] = df1['date_pulled'].str[:10]
#df1['date_pulled'] = df1['date_pulled'].str.replace('[^a-zA-Z0-9]', '', regex = True)
#df1['date_pulled'] = df1['date_pulled'].apply(lambda x: str(x))
#df1['date_pulled'] = df1['date_pulled'].apply(lambda x: x[0:2] + '/' + x[2:4] + '/' + x[4:8])

In [10]:
from datetime import timedelta, datetime

In [11]:
# formatting
df1['first_screening_date'] = pd.to_datetime(df1['first_screening_date'], format='%d/%m/%Y')
df1['first_screening_date'] = pd.to_datetime(df1['first_screening_date']).dt.date

ValueError: time data "- -" doesn't match format "%d/%m/%Y", at position 13. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [ ]:
df1['daily_takings'] = df1['daily_takings'].astype(float)
df1['daily_attendance'] = df1['daily_attendance'].astype(int)
df1['daily_rank'] = df1['daily_rank'].astype(int)
df1['total_takings'] = df1['total_takings'].astype(float)
df1['total_attendance'] = df1['total_attendance'].astype(int)

---

## Data manipulation

I decided to create new columns to get information I may need later on during the analysis.

In [ ]:
# calculating the number of days of screenings for each movie 
df1['screening_days'] = datetime.today().date() - df1['first_screening_date']
df1['screening_days'] = df1['screening_days'].map(lambda x: str(x)[:-14])

In [ ]:
df1['screening_days'] = df1['screening_days'].str.strip()
df1['screening_days'] = df1['screening_days'].replace('', '0')
df1['screening_days'] = df1['screening_days'].astype(int) + 1

In [ ]:
# creating a column to show the date the 'daily_takings' and 'daily_attendace' refer to
df1['date'] = df1['date_pulled'] - timedelta(days=1)
df1['date'] = pd.to_datetime(df1['date'])

In [ ]:
# creating a column showing the day of the week. it'll be useful for further analysis
df1['day_of_week'] = df1['date'].dt.day_name()

In [ ]:
df1['avg_ticket_price'] = df1['daily_takings'] / df1['daily_attendance']

In [ ]:
pd.set_option('display.max_rows', 8)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
df1

,daily_rank,title,first_screening_date,nation,distribution,daily_takings,daily_attendance,total_takings,total_attendance,date_pulled,screening_days,date,day_of_week,avg_ticket_price
0,1,C'E' Ancora Domani,2023-10-26,ITA,Vision Distribution,449991.06,66700,15231636.85,2202169,2023-11-17,38,2023-11-16,Thursday,6.75
1,2,Hunger Games - La Ballata Dell'Usignolo E Del ...,2023-11-15,USA,Medusa Film S.P.A.,238466.31,32469,584902.45,79311,2023-11-17,18,2023-11-16,Thursday,7.34
2,3,The Marvels,2023-11-08,USA,Walt Disney S.M.P. Italia,60914.35,8628,2260142.44,305291,2023-11-17,25,2023-11-16,Thursday,7.06
3,4,Thanksgiving,2023-11-16,USA,Eagle Pictures S.P.A.,41232.57,5709,41232.57,5709,2023-11-17,17,2023-11-16,Thursday,7.22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,7,La Guerra Dei Nonni,2023-11-30,ITA,Medusa Film S.P.A.,36998.22,5670,57696.33,8977,2023-12-02,3,2023-12-01,Friday,6.53
172,8,Il Silenzio Della Vendetta (Silent Night),2023-11-30,USA,Plaion Pictures,25036.05,3347,40976.77,5623,2023-12-02,3,2023-12-01,Friday,7.48
173,9,Palazzina Laf,2023-11-30,ITA,Bim Distrib. S.R.L.,24483.45,4001,59712.04,9469,2023-12-02,3,2023-12-01,Friday,6.12
174,10,The Old Oak,2023-11-16,GBR,Lucky Red Distrib.,16691.26,2554,743746.10,115442,2023-12-02,17,2023-12-01,Friday,6.54


---

## Export as .csv

In [ ]:
import os

In [ ]:
clean_file_name = 'cinetel_clean.csv'
clean_file_path = os.path.join(os.getcwd(), clean_file_name)

In [ ]:
df1.to_csv(clean_file_path, index=False)
print(f"Il DataFrame è stato esportato con successo in {clean_file_path}.")

Il DataFrame è stato esportato con successo in e:\data_analysis_python\cinetel\cinetel_clean.csv.


---

## Data Exploration

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
pd.set_option('display.max_rows', 8)

In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 160 entries, 0 to 174
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   daily_rank            160 non-null    int32         
 1   title                 160 non-null    object        
 2   first_screening_date  160 non-null    object        
 3   nation                160 non-null    object        
 4   distribution          160 non-null    object        
 5   daily_takings         160 non-null    float64       
 6   daily_attendance      160 non-null    int32         
 7   total_takings         160 non-null    float64       
 8   total_attendance      160 non-null    int32         
 9   date_pulled           160 non-null    object        
 10  screening_days        160 non-null    int32         
 11  date                  160 non-null    datetime64[ns]
 12  day_of_week           160 non-null    object        
 13  avg_ticket_price      160

In [ ]:
df1.describe()

,daily_rank,daily_takings,daily_attendance,total_takings,total_attendance,screening_days,date,avg_ticket_price
count,160.00,160.00,160.00,160.00,160.00,160.00,160,160.00
mean,5.50,158313.83,22557.75,3707488.39,530833.61,22.26,2023-11-23 12:00:00,6.86
min,1.00,8904.68,1216.00,11652.78,1470.00,3.00,2023-11-16 00:00:00,4.22
25%,3.00,19967.13,3367.25,344664.11,43204.00,12.25,2023-11-19 18:00:00,6.25
50%,5.50,44675.18,6743.00,1826453.13,266996.50,17.50,2023-11-23 12:00:00,6.78
75%,8.00,172906.10,24275.00,3427386.16,498059.75,26.50,2023-11-27 06:00:00,7.34
max,10.00,1544231.00,211764.00,25814847.27,3782616.00,87.00,2023-12-01 00:00:00,10.94
std,2.88,260445.33,36392.33,6201156.70,901833.31,15.70,NaN,1.04


In [ ]:
df1.isnull().sum()

daily_rank              0
title                   0
first_screening_date    0
nation                  0
                       ..
screening_days          0
date                    0
day_of_week             0
avg_ticket_price        0
Length: 14, dtype: int64

In [ ]:
df1.nunique()

daily_rank               10
title                    24
first_screening_date     13
nation                    5
                       ... 
screening_days           13
date                     16
day_of_week               7
avg_ticket_price        160
Length: 14, dtype: int64

In [ ]:
df1.corr(numeric_only=True)

,daily_rank,daily_takings,daily_attendance,total_takings,total_attendance,screening_days,avg_ticket_price
daily_rank,1.00,-0.64,-0.65,-0.51,-0.50,0.03,-0.14
daily_takings,-0.64,1.00,0.99,0.63,0.63,0.14,0.15
daily_attendance,-0.65,0.99,1.00,0.69,0.68,0.17,0.10
total_takings,-0.51,0.63,0.69,1.00,1.00,0.47,-0.14
total_attendance,-0.50,0.63,0.68,1.00,1.00,0.48,-0.16
screening_days,0.03,0.14,0.17,0.47,0.48,1.00,-0.43
avg_ticket_price,-0.14,0.15,0.10,-0.14,-0.16,-0.43,1.00


In [ ]:
df1.groupby('title').mean(numeric_only=True).sort_values(by = 'total_attendance', ascending= False)

,daily_rank,daily_takings,daily_attendance,total_takings,total_attendance,screening_days,avg_ticket_price
title,,,,,,,
C'E' Ancora Domani,1.12,679202.22,101348.94,21374500.95,3103305.19,38.00,6.53
Io Capitano,8.00,16027.86,3670.00,4226997.42,735242.60,87.00,4.38
Me Contro Te Il Film – Vacanze In Transilvania,10.00,37520.80,5383.00,4551425.30,674907.00,45.00,6.97
Killers Of The Flower Moon,9.67,23930.22,3229.33,5007558.46,673423.33,45.00,7.20
...,...,...,...,...,...,...,...
Picasso. Un Ribelle A Parigi. Storia Di Una Vita E Di Un Museo,5.00,27310.89,3306.33,51827.01,6312.00,6.00,8.24
La Guerra Dei Nonni,7.00,28713.61,4463.00,39062.67,6116.50,3.00,6.40
Il Silenzio Della Vendetta (Silent Night),8.50,20393.58,2790.50,28363.94,3928.50,3.00,7.27
Casanova Operapop - Il Film,9.00,12093.47,1256.33,25578.93,2715.33,6.00,9.62


In [ ]:
df2 = df1.groupby('title').mean(numeric_only=True).sort_values(by = 'total_attendance', ascending= False)
df2

,daily_rank,daily_takings,daily_attendance,total_takings,total_attendance,screening_days,avg_ticket_price
title,,,,,,,
C'E' Ancora Domani,1.12,679202.22,101348.94,21374500.95,3103305.19,38.00,6.53
Io Capitano,8.00,16027.86,3670.00,4226997.42,735242.60,87.00,4.38
Me Contro Te Il Film – Vacanze In Transilvania,10.00,37520.80,5383.00,4551425.30,674907.00,45.00,6.97
Killers Of The Flower Moon,9.67,23930.22,3229.33,5007558.46,673423.33,45.00,7.20
...,...,...,...,...,...,...,...
Picasso. Un Ribelle A Parigi. Storia Di Una Vita E Di Un Museo,5.00,27310.89,3306.33,51827.01,6312.00,6.00,8.24
La Guerra Dei Nonni,7.00,28713.61,4463.00,39062.67,6116.50,3.00,6.40
Il Silenzio Della Vendetta (Silent Night),8.50,20393.58,2790.50,28363.94,3928.50,3.00,7.27
Casanova Operapop - Il Film,9.00,12093.47,1256.33,25578.93,2715.33,6.00,9.62
